8 января 1986 года космический шаттл "Челленджер" взорвался при взлёте. Семь астронавтов, находившихся на борту, погибли. В ходе расследования причин катастрофы основной версией была неполадка с резиновыми уплотнительными кольцами в соединении с ракетными ускорителями. Для 23 предшествовавших катастрофе полётов "Челленджера" известны температура воздуха и появление повреждений хотя бы у одного из уплотнительных колец.

С помощью бутстрепа постройте 95% доверительный интервал для разности средних температур воздуха при запусках, когда уплотнительные кольца повреждались, и запусках, когда повреждений не было.

In [9]:
import pandas as pd
import numpy as np
import itertools
import random

In [10]:
path = 'https://raw.githubusercontent.com/chekhovana/courses/main/machine_learning/4_inference/2_%20ab_testing/data/2.6.3_challenger.txt'
data = pd.read_csv(path, sep='\t')
data.head()

,Unnamed: 0,Temperature,Incident
0,Apr12.81,18.9,0
1,Nov12.81,21.1,1
2,Mar22.82,20.6,0
3,Nov11.82,20.0,0
4,Apr04.83,19.4,0


In [11]:
sample_success = data[data.Incident == 0]['Temperature'].values
print(sample_success.shape)
sample_failure = data[data.Incident == 1]['Temperature'].values
print(np.mean(sample_success), np.mean(sample_failure))

(16,)
22.28125 17.614285714285717


In [12]:
def get_bootstrap_samples(data, n):
    return data[np.random.randint(0, len(data), size=(n, len(data)))]

In [13]:
def confint(data, alpha=0.05):
    return np.percentile(data, [100 * alpha / 2, 100 * (1 - alpha / 2)])

In [14]:
np.random.seed(0)
bootstrap_samples_success = get_bootstrap_samples(sample_success, 1000)
bootstrap_samples_failure = get_bootstrap_samples(sample_failure, 1000)
bootstrap_mean_success = np.mean(bootstrap_samples_success, axis=1)
bootstrap_mean_failure = np.mean(bootstrap_samples_failure, axis=1)
bootstrap_mean_diff = bootstrap_mean_success - bootstrap_mean_failure
print(confint(bootstrap_mean_success))
print(confint(bootstrap_mean_failure))
print(confint(bootstrap_mean_diff))

[21.06875  23.575625]
[14.5125     20.71607143]
[1.42299107 7.93861607]


Проверьте гипотезу об одинаковой средней температуре воздуха в дни, когда уплотнительный кольца повреждались, и дни, когда повреждений не было. Используйте перестановочный критерий и двустороннюю альтернативу. Чему равен достигаемый уровень значимости? 

In [15]:
def permutation_t_stat_ind(sample1, sample2):
    return np.mean(sample1) - np.mean(sample2)

In [16]:
def get_random_combinations(n1, n2, max_combinations):
    index = list(range(n1 + n2))
    indices = set([tuple(index)])
    for i in range(max_combinations - 1):
        np.random.shuffle(index)
        indices.add(tuple(index))
    return [(index[:n1], index[n1:]) for index in indices]

In [17]:
def permutation_zero_dist_ind(sample1, sample2, max_combinations = None):
    joined_sample = np.hstack((sample1, sample2))
    n1 = len(sample1)
    n = len(joined_sample)
    
    if max_combinations:
        indices = get_random_combinations(n1, len(sample2), max_combinations)
    else:
        indices = [(list(index), list(filter(lambda i: i not in index, range(n)))) \
                    for index in itertools.combinations(range(n), n1)]
    
    distr = [joined_sample[list(i[0])].mean() - joined_sample[list(i[1])].mean() \
             for i in indices]
    return distr

In [18]:
def permutation_test(sample, mean, max_permutations = None, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    t_stat = permutation_t_stat_ind(sample, mean)
    
    zero_distr = permutation_zero_dist_ind(sample, mean, max_permutations)
    
    if alternative == 'two-sided':
        return sum([1. if abs(x) >= abs(t_stat) else 0. for x in zero_distr]) / len(zero_distr)
    
    if alternative == 'less':
        return sum([1. if x <= t_stat else 0. for x in zero_distr]) / len(zero_distr)

    if alternative == 'greater':
        return sum([1. if x >= t_stat else 0. for x in zero_distr]) / len(zero_distr)

In [20]:
np.random.seed(0)
random.seed
print('pvalue', permutation_test(sample_success, sample_failure, max_permutations=10000))

pvalue 0.007
